In [13]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import warnings
import pprint
import time
warnings.simplefilter('ignore')

#ファイルのインポート
import evaluation as ev
import control_structure as cs
import race
import table as tb
import init_table as init
import creating_test as test
import os_operation as _os_

RESULTS_FILE_NAME='追加実験②' #.csvつけない

# インポートする dir
path=''

# エクスポートする dir
results_path='results②/'



heading_results=('SYN','Task','Buffer','TestCase','SVAR','Def-Use','Plnv','Race')
heading=('Thread','Port','Event','Index')
pre_results=pd.DataFrame({},columns=heading_results)
new_results=pd.DataFrame({},columns=heading_results)


RECEIVE_SHEET_NAME='追加実験/recv-task-6-buffer-1-loop1.csv'
SEND_SHEET_NAME='追加実験/send-task-6-buffer-1-loop1.csv'



Qs=[pd.DataFrame({})]
Qr=[pd.DataFrame({})]
Qs[0]=pd.read_csv(SEND_SHEET_NAME,names=heading)
Qr[0]=pd.read_csv(RECEIVE_SHEET_NAME,names=heading)


init_tables=init.init_main(Qr,Qs)
Qr=init_tables['Qr']
Qs=init_tables['Qs']
Qr_unique=init_tables['Qr_unique']
Qs_unique=init_tables['Qs_unique']

for way in range(2):
    if way==0:
        t_way=2
        combination=0 #0の場合は既存研究の方法、1の場合は組み合わせを考慮した手法
        ev_table=pre_results
    elif way==1:
        t_way=2
        combination=1
        ev_table=new_results
        Qr=Qr[:1]
        Qs=Qs[:1]
        Qr_unique=init_tables['Qr_unique']
        Qs_unique=init_tables['Qs_unique']
    else:
        print('プログラムを終了します') # shoudn't reach here
        break


    number=0 # 実験の回数
    check_digit=0

    race_set={}


    start = time.time()
    print("Creating race set",end='')
    race_set=race.creating_race_set(Qr,Qs,Qs_unique,race_set,combination)
    elapsed_time = time.time() - start
    print ("\nCreating race set took:{:.4g}".format(elapsed_time) + "[sec]")
    #pprint.pprint((race_set))


    start = time.time()
    print("Creating race table",end='')
    table=tb.construct_race_table(Qs,Qr,Qs_unique,Qr_unique,race_set,t_way)
    if t_way>1:
        table=tb.expand_table(Qr,Qs,race_set,table,t_way,Qs_unique)
    elapsed_time = time.time() - start
    print ("\nCreating race table took:{:.4g}".format(elapsed_time) + "[sec]")
    table=table.astype('int64')

    check_digit=1

    start = time.time()
    results=test.create_new_testcase(number,check_digit,Qr,Qs,table,race_set,Qr_unique,Qs_unique)
    Qr=results['recv']
    Qs=results['send']
    elapsed_time = time.time() - start
    print ("Creating test case took:{:.4g}".format(elapsed_time) + "[sec]")
    print('The number of Test Case is {}.'.format(len(Qr)))


    evaluation=ev.evaluate(Qr,Qs,race_set)
    Q=pd.concat([Qs[0],Qr[0]]).sort_index()
    display(table)
    if way==0:
        pre_results=pre_results.append(evaluation,ignore_index=True)
    elif way==1:
        new_results=new_results.append(evaluation,ignore_index=True)
    else:
        print('error')

new_results=new_results.sort_values(by=['Task'])
new_results=new_results.drop(columns='SYN')
pre_results=pre_results.sort_values(by=['Task'])
results_concat=pd.concat([pre_results,new_results],axis=1)
_os_.save_file(RESULTS_FILE_NAME,results_path,results_concat)


100%|██████████| 22/22 [00:00<00:00, 154.93it/s]


finish initialize
Creating race set
Creating race set took:0.142[sec]
Creating race table................
Expanding table
......................................................................................................................................................................................................................................................................................................................................................................................
Creating race table took:41.37[sec]
.................................................................................................................................................................................Creating test case took:33.82[sec]
The number of Test Case is 178.
Evaluating


,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,0,1,1,1,1,1,1,2,0
1,0,2,2,2,2,1,0,2,0
2,0,3,3,3,3,1,2,2,0
3,1,0,1,2,3,2,0,0,1
4,1,1,0,3,1,0,0,1,2
...,...,...,...,...,...,...,...,...,...
172,0,0,0,0,0,0,0,3,3
173,0,0,0,0,0,0,0,0,3
174,0,0,0,0,0,0,0,1,3
175,0,0,0,0,0,0,0,2,3


Creating race set
Creating race set took:0.17[sec]
Creating race table.........
Expanding table
......................................................................................................................................................................................................................................................................................................................
Creating race table took:20.77[sec]
........................................................................................................................................Creating test case took:25.69[sec]
The number of Test Case is 137.
Evaluating


,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,0,1,1,1,1,0,1,1,1
1,0,2,2,2,0,0,1,1,1
2,1,0,1,2,1,1,0,1,0
3,1,1,0,0,0,1,1,1,1
4,1,2,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
131,0,0,0,0,0,0,0,0,2
132,0,0,0,0,0,0,1,0,2
133,0,0,0,0,0,0,0,2,2
134,0,0,0,0,0,0,0,0,2


In [12]:
Qr_unique

,ID,Thread,Port,Event,Index,cstruct
0,r1,T6,16045988,recv,1,
1,r2,T6,16045988,recv,2,"[r1, s1]"
2,r3,T1,16045988,recv,1,
3,r4,T2,16045988,recv,1,
4,r5,T3,16045988,recv,1,
5,r6,T4,16045988,recv,1,
6,r7,T5,16045988,recv,1,
7,r8,T6,16045988,recv,3,"[s2, r2, s1, r1]"
8,r9,T1,16045988,recv,2,"[s3, r3]"
9,r10,T2,16045988,recv,2,"[r4, s4]"


In [5]:
Qs_unique

,ID,Thread,Port,Event,Index,cstruct
0,s1,T1,p2,send,1,
1,s2,T3,p2,send,1,
2,s3,T4,p2,send,1,
3,s4,T1,p2,send,2,[s1]
4,s5,T1,p2,send,3,"[s1, s4]"


In [ ]:
pair_event

In [16]:
Qs[0].iloc[i]

ID            s26
Thread         T5
Port      9682268
Event        send
Index           9
KEY          T5-9
Name: 25, dtype: object

In [14]:
Qr_temp

,ID,Thread,Port,Event,Index,KEY
0,r1,T3,9682268,recv,1,T3-1
1,r2,T3,9682268,recv,2,T3-2
4,r5,T3,9682268,recv,3,T3-3
7,r8,T3,9682268,recv,4,T3-4
10,r11,T3,9682268,recv,5,T3-5
13,r14,T3,9682268,recv,6,T3-6
16,r17,T3,9682268,recv,7,T3-7
19,r20,T3,9682268,recv,8,T3-8
22,r23,T3,9682268,recv,9,T3-9


In [20]:
for j in range(len(Qr_temp)):
    s_dash=Qs[0].loc[Qr_temp.iloc[j].name]
    if s_dash.ID not in Qs_unique[Qs_unique.ID==ID].iloc[0].cstruct:
        Qr_temp.iloc[j]
    else:
        print('{} in {}'.format(s_dash.ID,Qs_unique[Qs_unique.ID==ID].iloc[0].cstruct))

s2 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s5 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s8 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s11 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s14 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s17 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s20 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']
s23 in ['s14', 's5', 's2', 's11', 's8', 's23', 's17', 's20']


In [18]:
Qs_unique

,ID,Thread,Port,Event,Index,cstruct
0,s1,T4,9682268,send,1,
1,s2,T5,9682268,send,1,
2,s3,T6,9682268,send,1,
3,s4,T4,9682268,send,2,[s1]
4,s5,T5,9682268,send,2,[s2]
5,s6,T6,9682268,send,2,[s3]
6,s7,T4,9682268,send,3,"[s1, s4]"
7,s8,T5,9682268,send,3,"[s2, s5]"
8,s9,T6,9682268,send,3,"[s3, s6]"
9,s10,T4,9682268,send,4,"[s1, s4, s7]"
